5 a)

In [12]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import collections
import sys
sys.path.append('/content/drive/MyDrive/cs229/problem-sets/PS2/src') # Python interpreter searches for modules/packages to import from a list of directories called 'PYTHONPATH', which is added to here
import util # import the entire '.py' file, executing any top-level code, and copying all functions, variables, and classes
import svm
sys.path.remove('/content/drive/MyDrive/cs229/problem-sets/PS2/src')  # remove added path from 'PYTHONPATH'
%cd /content/drive/MyDrive/cs229/problem-sets/PS2/src/


def get_words(message):
    """Get the normalized list of words from a message string.
    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.
    Args:
        message: A string containing an SMS message
    Returns:
       The list of normalized words from the message.
    """
    # *** START CODE HERE ***
    normalized = message.lower()
    normalized = normalized.split()
    return normalized
    # *** END CODE HERE ***

def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.
    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.
    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.
    Args:
        messages: A list of strings containing SMS messages
    Returns:
        A python dict mapping words to integers.
    """
    # *** START CODE HERE ***
    frequency = dict()
    for message in messages:
      normalized = get_words(message)
      for word in normalized:
        if word in frequency:
          frequency[word] += 1
        else:
          frequency[word] = 1
    common_dictionary = dict()
    index = 0
    for word in frequency:
      if frequency[word] >= 5:
        common_dictionary[word] = index
        index += 1
    return common_dictionary
    # *** END CODE HERE ***

def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.
    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each
    message and each column should correspond to a word.
    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.
    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.
    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    word_presence = np.zeros((len(messages), len(word_dictionary)))
    message_index = -1
    for message in messages:
      message_index += 1
      for word in get_words(message):
        if word in word_dictionary:
          word_presence[message_index, word_dictionary[word]] += 1
    return word_presence
    # *** END CODE HERE ***

class naive_bayes_model:
  def __init__(self, dictionary_length):
    self.phi_1 = np.zeros(dictionary_length)
    self.phi_0 = np.zeros(dictionary_length)
    self.phi = 0

def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.
    This function should fit a Naive Bayes model given a training matrix and labels.
    The function should return the state of that model.
    Feel free to use whatever datatype you wish for the state of the model.
    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data
    Returns: The trained model
    """
    # *** START CODE HERE ***
    dictionary_length = matrix.shape[1]
    model = naive_bayes_model(dictionary_length)
    phi_0_numerator = np.sum(matrix[labels == 0], axis=0)
    weighted_zero_count = np.sum(phi_0_numerator)
    model.phi_0 = (phi_0_numerator + 1) / (weighted_zero_count + dictionary_length)
    phi_1_numerator = np.sum(matrix[labels == 1], axis=0)
    weighted_one_count = np.sum(phi_1_numerator)
    model.phi_1 = (phi_1_numerator + 1) / (weighted_one_count + dictionary_length)
    one_count = np.sum(labels == 1)
    sample_size = matrix.shape[0]
    model.phi = one_count / sample_size
    return model
    # *** END CODE HERE ***

def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.
    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.
    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts
    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    log_phi_1 = np.log(model.phi_1)
    log_phi_0 = np.log(model.phi_0)
    log_one_probabilities = matrix @ log_phi_1.T
    log_zero_probabilities = matrix @ log_phi_0.T
    log_p_x_1 = log_one_probabilities + np.log(model.phi)
    log_p_x_0 = log_zero_probabilities + np.log(1 - model.phi)
    return np.where(log_p_x_1 >= log_p_x_0, 1, 0)
    # *** END CODE HERE ***

def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.
    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.
    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids
    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    indicativity = np.log(model.phi_1)-np.log(model.phi_0)
    sorted_indices = np.argsort(indicativity)
    top_5_indices = sorted_indices[-5:]
    return top_5_indices.tolist()
    # *** END CODE HERE ***

def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.
    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.
    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    maximum_accuracy = 0
    best_radius = radius_to_consider[0]
    for i in range(len(radius_to_consider)):
      prediction = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius_to_consider[i])
      accuracy = np.mean(prediction == val_labels)
      if accuracy > maximum_accuracy:
        maximum_accuracy = accuracy
        best_radius = radius_to_consider[i]
    return best_radius
    # *** END CODE HERE ***

def main():
    train_messages, train_labels = util.load_spam_dataset('../data/ds6_train.tsv')
    val_messages, val_labels = util.load_spam_dataset('../data/ds6_val.tsv')
    test_messages, test_labels = util.load_spam_dataset('../data/ds6_test.tsv')
    dictionary = create_dictionary(train_messages)
    util.write_json('./output/p06_dictionary', dictionary)
    train_matrix = transform_text(train_messages, dictionary)
    np.savetxt('./output/p06_sample_train_matrix', train_matrix[:100,:])
    val_matrix = transform_text(val_messages, dictionary)
    test_matrix = transform_text(test_messages, dictionary)
    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)
    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)
    np.savetxt('./output/p06_naive_bayes_predictions', naive_bayes_predictions)
    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)
    print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))
    top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)
    index_to_word = {v: k for k, v in dictionary.items()}
    top_5_words_listed = []
    for i in range(5):
      top_5_words_listed.append(index_to_word[top_5_words[i]])
    print('The top 5 indicative words for Naive Bayes are: ', top_5_words_listed)
    util.write_json('./output/p06_top_indicative_words', top_5_words_listed)
    optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])
    util.write_json('./output/p06_optimal_radius', optimal_radius)
    print('The optimal SVM radius was {}'.format(optimal_radius))
    svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)
    svm_accuracy = np.mean(svm_predictions == test_labels)
    print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))


if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cs229/problem-sets/PS2/src
Naive Bayes had an accuracy of 0.978494623655914 on the testing set
The top 5 indicative words for Naive Bayes are:  ['urgent!', 'tone', 'prize', 'won', 'claim']
The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9695340501792115 on the testing set
